In [1]:
%load_ext aiida
%aiida 

Loaded AiiDA DB environment - profile name: demo.

In [2]:
from pymatgen import get_structure_from_mp
import aiida.orm as orm
from aiida_user_addons.common.inputset.vaspsets import VASPInputSet
from aiida_user_addons.tools.dryrun import dryrun_vasp, get_jobscheme
from aiida_user_addons.vworkflows.relax import VaspRelaxWorkChain, RelaxOptions

Get the SrTiO3 structure from pymatgen

In [3]:
sto = StructureData(pymatgen=get_structure_from_mp("mp-5229"))

Setup the input for a `VaspCalculation`. 

Note: normally it is more common to use `VaspWorkChain` or `VaspRelaxWorkChain` for running the calculations.

In [4]:
inputset = VASPInputSet("UCLRelaxSet", sto, overrides={'encut': 300})
potential = DataFactory('vasp.potcar').get_potcars_from_structure(
                structure=sto,
                family_name='PBE.54',
                mapping=inputset.get_pp_mapping())


builder = CalculationFactory('vasp.vasp').get_builder()
builder.structure = sto
builder.parameters = inputset.get_input_dict(raw_python=False)
builder.potential = potential
builder.metadata.options = {
    'resources': {'num_machines': 1, 'tot_num_mpiprocs': 4}
}
builder.kpoints = inputset.get_kpoints(0.05)
builder.code = orm.load_code("vasp@localhost")


In [5]:
dryrun_results = dryrun_vasp(builder, '/home/bonan/appdir/VASP/vasp.5.4.4/bin/vasp_std')

In [6]:
dryrun_results

{'POTCARS': ['PAW_PBE Sr_sv 07Sep2000',
  'PAW_PBE Ti_pv 07Sep2000',
  'PAW_PBE O 08Apr2002'],
 'num_kpoints': 20,
 'num_bands': 26,
 'num_plane_waves': 13824,
 'NGX': 24,
 'NGY': 24,
 'NGZ': 24,
 'kpoints_and_weights': [[0.0, 0.0, 0.0, 0.005],
  [0.16666667, 0.0, 0.0, 0.028],
  [0.33333333, 0.0, 0.0, 0.028],
  [0.5, 0.0, 0.0, 0.014],
  [0.16666667, 0.16666667, 0.0, 0.056],
  [0.33333333, 0.16666667, 0.0, 0.111],
  [0.5, 0.16666667, 0.0, 0.056],
  [0.33333333, 0.33333333, 0.0, 0.056],
  [0.5, 0.33333333, 0.0, 0.056],
  [0.5, 0.5, 0.0, 0.014],
  [0.16666667, 0.16666667, 0.16666667, 0.037],
  [0.33333333, 0.16666667, 0.16666667, 0.111],
  [0.5, 0.16666667, 0.16666667, 0.056],
  [0.33333333, 0.33333333, 0.16666667, 0.111],
  [0.5, 0.33333333, 0.16666667, 0.111],
  [0.5, 0.5, 0.16666667, 0.028],
  [0.33333333, 0.33333333, 0.33333333, 0.037],
  [0.5, 0.33333333, 0.33333333, 0.056],
  [0.5, 0.5, 0.33333333, 0.028],
  [0.5, 0.5, 0.5, 0.005]],
 'plane_waves_min_max': [739.0, 708.0],
 'max_ram_

In [9]:
get_kpar_ncore(builder, 24, '/home/bonan/appdir/VASP/vasp.5.4.4/bin/vasp_std', cpus_per_node=24)

In [6]:
get_optimum_parallelisation(builder, 24, '/home/bonan/appdir/VASP/vasp.5.4.4/bin/vasp_std', cpus_per_node=24, ncore_strategy='balance')

09/27/2020 05:02:45 PM <3920> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [INFO] Broadcasting state change of 277995566052394404462360341832778824635: state_changed.created.running
09/27/2020 05:02:45 PM <3920> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [INFO] Broadcasting state change of 277995566052394404462360341832778824635: state_changed.running.finished


(4, 2)

Here the number of kpoints and number of bands have been obtained, we can use them to work out the best `KPAR` value.  
The number of bands and plane wave can also be taken account to check if the calculation will be over-parallelised.